## Import Scraped Data from Reddit

In [1]:
import pandas as pd
scraped_data1 = pd.read_csv("./data/REDDIT_POSTS_Israel_Hamas_Palestine_geopolitics_worldnews_news.csv")
scraped_data2 = pd.read_csv("./data/REDDIT_POSTS_Israel_Hamas_Palestine_Israel_IsraelPalestine_Palestine_Politics_Conservative.csv")

In [2]:
raw_data = pd.concat([scraped_data1, scraped_data2],axis=0)

## Remove Posts that don't contain text

In [3]:
print(raw_data["post_body"].unique())

[nan
 "Company: Windranger Labs\r\n\r\nLocation: Afghanistan Albania Algeria Andorra Angola Antarctica Armenia Austria Azerbaijan Bahrain Belarus Belgium Benin Bosnia and Herzegovina Botswana Bouvet Island Bulgaria Burkina Faso Burundi Cabo Verde Cameroon Central African Republic Chad Comoros Congo Congo, The Democratic Republic of the Cook Islands Croatia Curaçao Cyprus Czechia Côte d'Ivoire Denmark Djibouti Egypt Equatorial Guinea Eritrea Estonia Eswatini Ethiopia Faroe Islands Finland France French Guiana French Southern Territories Gabon Gambia Georgia Germany Ghana Gibraltar Greece Greenland Guadeloupe Guernsey Guinea Guinea-Bissau Heard Island and McDonald Islands Holy See (Vatican City State) Hungary Iceland Iran Iraq Ireland Isle of Man Israel Italy Jersey Jordan Kazakhstan Kenya Kuwait Kyrgyzstan Latvia Lebanon Lesotho Liberia Libya Liechtenstein Lithuania Luxembourg Madagascar Malawi Mali Malta Martinique Mauritania Mauritius Mayotte Moldova Monaco Montenegro Morocco Mozambiq

In [4]:
print(len(raw_data))

5924


In [5]:
raw_data["valid_data"] = raw_data["post_body"].apply(lambda x: isinstance(x,str))

In [6]:
data = raw_data[raw_data["valid_data"] == True]

In [7]:
print(data["post_body"].unique())

["Company: Windranger Labs\r\n\r\nLocation: Afghanistan Albania Algeria Andorra Angola Antarctica Armenia Austria Azerbaijan Bahrain Belarus Belgium Benin Bosnia and Herzegovina Botswana Bouvet Island Bulgaria Burkina Faso Burundi Cabo Verde Cameroon Central African Republic Chad Comoros Congo Congo, The Democratic Republic of the Cook Islands Croatia Curaçao Cyprus Czechia Côte d'Ivoire Denmark Djibouti Egypt Equatorial Guinea Eritrea Estonia Eswatini Ethiopia Faroe Islands Finland France French Guiana French Southern Territories Gabon Gambia Georgia Germany Ghana Gibraltar Greece Greenland Guadeloupe Guernsey Guinea Guinea-Bissau Heard Island and McDonald Islands Holy See (Vatican City State) Hungary Iceland Iran Iraq Ireland Isle of Man Israel Italy Jersey Jordan Kazakhstan Kenya Kuwait Kyrgyzstan Latvia Lebanon Lesotho Liberia Libya Liechtenstein Lithuania Luxembourg Madagascar Malawi Mali Malta Martinique Mauritania Mauritius Mayotte Moldova Monaco Montenegro Morocco Mozambique Na

In [8]:
print(len(data))

1795


## Pre-process the text of Posts

In [9]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/local/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /usr/local/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
def preprocess(text_series):
        processed_text = []
        for text in text_series:
            if isinstance(text,str) == False:
                processed_text.append([]);
                continue
            # remove contractions
            text = text.lower()
            text = re.sub(r'\W', ' ',text)
            tokens = word_tokenize(text)
            
            # Remove stopwords - except for no and not to capture negation
            stop_words = set(stopwords.words('english'))
            stop_words.remove("no")
            stop_words.remove("not")
            filtered_tokens = [word for word in tokens if word not in stop_words]

            # Lemmatization
            lemmatizer = WordNetLemmatizer()
            lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

#             print("Original Text:", text)
#             print("Filtered Tokens:", filtered_tokens)
#             print("Lemmatized Tokens:", lemmatized_tokens)
            filtered_input = ' '.join(lemmatized_tokens)
            #print("Joined sentence:" , filtered_input)
            processed_text.append(filtered_input)
            
        return processed_text

In [11]:
pp_input = preprocess(data["post_body"])
data["pre-processed"] = pp_input

/var/folders/jh/7_7blzs94bs2rrfl07z2j84r0000gn/T/ipykernel_1508/3825621500.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["pre-processed"] = pp_input


## Apply Tf-IDF Encoding to the Posts
Note: the Tf-IDF Vectorizer needs to fit across all of the documents in order to determine what are the unique terms within each document

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
def encode_tfidf(pp_input):
    # Create the vectorizer and fit it to the documents
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(pp_input)
    # Print the feature names and the resulting vectors
    print(vectorizer.get_feature_names_out())
    print(tfidf_matrix.toarray())
            
    return tfidf_matrix.toarray()

In [14]:
tfidf_matrix = encode_tfidf(data["pre-processed"])
print(tfidf_matrix)

['00' '000' '0000' ... '한국' '호황' 'ﬁnance']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Example documents
documents = data["pre-processed"].values

# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Generate the TF-IDF matrix
tfidf_matrix = vectorizer.fit_transform(documents)

# Get feature names to access the terms used in the TF-IDF matrix
feature_names = vectorizer.get_feature_names_out()

top_terms_list = []
# Iterate through each document and find terms with the highest TF-IDF scores
for doc_idx, doc in enumerate(tfidf_matrix):
    # Convert the sparse matrix row to a dense array
    dense_doc = doc.toarray().flatten()
    
    # Sort indices of feature values in descending order of their TF-IDF scores
    sorted_indices = np.argsort(dense_doc)[::-1]
    
    # Get the terms corresponding to the top n feature indices
    top_terms = [feature_names[idx] for idx in sorted_indices[:50]]  # Adjust the number to get more or fewer terms
    
    # Print the most important words for each document
    print(f"Document {doc_idx + 1}: {top_terms}")
    top_terms_list.append(top_terms)


Document 1: ['island', 'saint', 'guinea', 'republic', 'french', 'congo', 'sudan', 'south', 'united', 'ahr0chm6ly9oaw1hbgf5yxmuyxbwl2nvbxbhbmllcy93aw5kcmfuz2vylwxhynmvam9icy96a3zhbglkyxrvci1yzxnlyxjjac1hbmfsexn0', 'windranger', 'lab', 'arab', 'part', 'verde', 'marino', 'svalbard', 'miquelon', 'pierre', 'gabon', 'seychelles', 'burundi', 'helena', 'réunion', 'cabo', 'tome', 'tristan', 'barthélemy', 'cameroon', 'mayotte', 'mozambique', 'sao', 'djibouti', 'åland', 'eswatini', 'guernsey', 'malawi', 'mayen', 'comoros', 'cunha', 'maarten', 'curaçao', 'gambia', 'principe', 'leone', 'togo', 'antarctica', 'bouvet', 'faroe', 'sint']
Document 2: ['classmate', 'nonsense', 'morally', 'knowing', 'people', 'subject', 'using', 'community', 'example', 'not', 'flared', 'islamophobic', 'innumerable', 'anyone', 'qur', 'crop', 'spewing', 'bigoted', 'remotely', 'without', 'reel', 'islamophobia', 'lot', 'judgment', 'ought', 'dramatically', 'concluded', 'balfour', 'suspended', 'affected', 'overwhelming', 'ill',

Document 400: ['hybrid', 'dictatorship', 'authoritarian', 'turkey', 'democracy', 'regime', 'democratic', 'rather', 'example', 'make', 'country', 'education', 'educational', 'edward', 'een', 'eeas', 'educating', 'eerder', 'eerily', 'eez', 'ef1bd6044cc680ac1ba75baddfbb1b0985293191', 'eff', 'effect', 'educatiom', 'ﬁnance', 'educated', 'editorial', 'edifice', 'edit', 'edit2', 'edited', 'editing', 'edition', 'editor', 'editorializing', 'effective', 'editorially', 'edits', 'edl', 'edovit__o8a', 'edt', 'edu', 'educate', 'effected', 'efficiently', 'effectively', 'einfomax', 'ehktbxrm7o', 'ehud', 'eid']
Document 401: ['occrp', 'google', 'x200b', 'reporting', 'related', 'speak', 'jitsi', 'viewform', 'storytelling', '1faipqlsffcxx1a9qjnbukns3i9uabmlxkf8uptc16c2jac_wyo99s6w', 'form', 'investigative', 'arrange', 'amsterdam', 'whichever', 'russia', 'knowledgable', 'feedback', 'routinely', 'email', 'virtual', 'doc', 'implement', 'fill', 'corruption', 'geopolitics', 'coverage', 'prefer', 'minute', 'or

Document 801: ['released', 'hostage', 'terrorist', 'leverage', 'ratio', 'insane', 'fully', 'multiple', 'happen', 'looking', 'someone', 'past', 'current', 'number', 'anything', 'situation', 'really', 'need', 'side', 'much', 'support', 'many', 'palestinian', 'israel', 'educatiom', 'educating', 'edits', 'educated', 'education', 'educate', 'educational', 'edward', 'edu', 'edt', 'edovit__o8a', 'eeas', 'edl', 'editor', 'editorially', 'editorializing', 'ecuador', 'ed', 'einstein', 'edelman', 'eden', 'edge', 'edifice', 'edit', 'edit2', 'edited']
Document 802: ['convince', 'created', 'palestinian', 'hamas', 'though', 'trying', 'claim', 'american', 'fact', 'pro', 'even', 'israel', 'educated', 'educating', 'educatiom', 'education', 'educational', 'ﬁnance', 'edward', 'eeas', 'educate', 'eerder', 'eerily', 'eez', 'ef1bd6044cc680ac1ba75baddfbb1b0985293191', 'eff', 'een', 'edt', 'edu', 'editing', 'edelman', 'eden', 'edge', 'edifice', 'edit', 'edit2', 'edited', 'edition', 'effected', 'editor', 'editor

Document 1208: ['identity', 'national', 'self', 'unique', 'state', 'determination', 'ever', 'argument', 'palestinian', 'existed', 'fake', 'no', 'group', 'british', 'asking', 'culture', 'occupation', 'rule', 'balush', 'accuire', 'ancestory', 'deligimimize', 'romani', 'kenyan', 'disqualify', 'shallow', 'simply', 'case', 'laughable', 'practiced', 'kurdish', 'nation', 'characteristic', 'strongest', 'use', 'history', 'dictate', 'statehood', 'recorded', 'assumption', 'indian', 'exercise', 'race', 'looked', 'colonial', 'deny', 'surrounding', 'accurate', 'country', 'independence']
Document 1209: ['refugee', 'resolution', '1957', 'fabricated', 'conference', 'quote', 'serve', 'palestinian', 'return', 'homs', 'desecration', 'disconfirming', 'massa', 'verifying', 'case', 'veracity', 'overview', 'dershowitz', 'help', 'treason', 'quoted', 'beirut', 'damning', 'destabilize', 'ensuring', 'dig', 'weight', 'column', 'confident', 'fifth', 'jewishvirtuallibrary', 'regarded', 'annihilate', 'israel', 'would

Document 1600: ['people', 'sure', 'outsider', 'likely', 'treating', 'think', 'wrong', 'least', 'way', 'excuse', 'get', 'center', 'extremist', 'war', 'pro', 'not', 'online', 'seeing', 'like', 'irks', 'humanize', 'baked', 'nitpick', 'ck', 'tad', 'want', 'le', 'gripe', 'lynched', 'started', 'living', 'word', 'matter', 'depressing', 'irl', 'put', 'logically', 'ash', 'palestine', 'alt', 'optimistic', 'backlash', 'dust', 'chest', 'luck', 'stress', 'influenced', 'hypothetical', '99', 'restraint']
Document 1601: ['bibi', 'likud', 'hamas', 'election', 'israeli', 'knowing', 'tunnel', 'unified', 'stay', 'power', 'rejection', 'violent', 'gaza', 'attack', 'violence', 'oppose', 'effectively', 'free', 'colonialism', 'blood', 'east', 'end', 'civilian', 'middle', 'strategy', 'gazans', 'antisemitism', 'party', 'reignite', 'deputizing', 'pyramid', 'rabim', 'instigate', 'secures', 'mlk', 'israel', 'oct', 'entire', 'unharmed', 'decimate', 'un', 'demonic', 'communicate', 'enforced', 'better', 'recruitment',

In [16]:
data["top_terms"] = top_terms_list

/var/folders/jh/7_7blzs94bs2rrfl07z2j84r0000gn/T/ipykernel_1508/1528530591.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["top_terms"] = top_terms_list


## Apply Word2Vec Encodings

In [18]:
from gensim.models import Word2Vec
import gensim.downloader as api
from gensim.models import KeyedVectors

In [21]:
model = KeyedVectors.load('./word2Vec_mod12.model')

In [30]:
def encode_w2v(pp_input,mod):
    # Load pre-trained Word2Vec model or train your own
    model = mod  # This loads a large model, which might take time and memory
    # Example words
    all_word_vectors = []
    print(pp_input)
    for list_of_common_words in pp_input:
        word_vectors = []
        words = list_of_common_words
        # Get vectors for words
        word_vectors = [model[word] for word in words if word in model]
        all_word_vectors.append(word_vectors)
            
    return all_word_vectors


In [32]:
processed_values = encode_w2v(data["top_terms"],model)
data["encoded"] = processed_values

2       [island, saint, guinea, republic, french, cong...
3       [classmate, nonsense, morally, knowing, people...
4       [hell, actually, like, ﬁnance, eeas, educated,...
9       [david, meshugga, king, crazy, achish, god, pa...
15      [nanobytesinc, watch, away, bathtub, coke, dri...
                              ...                        
3573    [cupe, trudy, fascinating, labour, huh, embrac...
3574    [mod, weird, trojan, upvote, benign, greyed, c...
3584    [flair, conservative, user, apply, discord, mo...
3662    [ﬁnance, edward, edu, educate, educated, educa...
3717    [good, really, much, thing, not, edward, educa...
Name: top_terms, Length: 1795, dtype: object


/var/folders/jh/7_7blzs94bs2rrfl07z2j84r0000gn/T/ipykernel_1508/3496911535.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["encoded"] = processed_values


In [33]:
print(data.columns)

Index(['subreddit', 'post_id', 'title', 'post_body', 'post_author', 'url',
       'post_permalink', 'num_comments', 'post_created', 'post_score',
       'post_distinguished', 'original_content', 'upvote_ratio', 'flair_text',
       'scrape_time', 'valid_data', 'pre-processed', 'top_terms', 'encoded'],
      dtype='object')


In [35]:
def average_vectors(vectors):
    # Ensure that the list is not empty
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        # Return a zero vector if no vectors available
        # Make sure to match the dimension of your word vectors
        return np.zeros(shape=(300,))

In [37]:
data["mean_encoded"] = data["encoded"].apply(average_vectors)

/var/folders/jh/7_7blzs94bs2rrfl07z2j84r0000gn/T/ipykernel_1508/2675552895.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["mean_encoded"] = data["encoded"].apply(average_vectors)


## Prepare DataFrame for Clustering Analysis

In [40]:
expanded_df = pd.DataFrame(data['mean_encoded'].tolist(), index=data.index)
expanded_df.columns = [f"Column_{i}" for i in range(expanded_df.shape[1])]

## Use the Elbow Method to determine the number of clusters

In [42]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans


# Calculating WSS for different number of clusters
wss = []
for i in range(1, 30):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(expanded_df.to_numpy())
    wss.append(kmeans.inertia_)

# Plotting the elbow curve
plt.plot(range(1, 30), wss,"b*")
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WSS')
plt.show()

I will select 8 clusters for the Elbow of the plot

In [43]:
# Number of clusters
k = 8

# Creating a KMeans object
kmeans = KMeans(n_clusters=k, random_state=42)

# Fitting the model on your DataFrame
kmeans.fit(expanded_df)

# Getting the centroids of each cluster
centroids = kmeans.cluster_centers_
print("Centroids of each cluster:")
for i, centroid in enumerate(centroids):
    print(f"Cluster {i}: {centroid}")

# Assigning each sample to a cluster
cluster_assignments = kmeans.labels_

# Printing the indices of points for each cluster
clusters = {i: [] for i in range(k)}
for index, label in enumerate(cluster_assignments):
    clusters[label].append(index)

for cluster, indices in clusters.items():
    print(f"Cluster {cluster} includes indices: {len(indices)}")

/Users/ryanbergmac/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Centroids of each cluster:
Cluster 0: [ 0.04752577  0.02933107  0.02256406  0.08713864 -0.07239377  0.01258477
  0.05895094 -0.06327645  0.09451421  0.06117295 -0.02503642 -0.11881706
 -0.04238579  0.04193179 -0.09881167  0.08258613  0.02741298  0.09403496
  0.00582475 -0.08711408  0.02016625  0.04896217  0.04789954 -0.00374029
  0.0247171  -0.03065204 -0.06857549  0.04583971  0.04346451 -0.03381728
 -0.00515596 -0.00635249 -0.04878174  0.0038342   0.00024526 -0.013921
  0.06159263  0.01211688  0.04053737  0.05837582  0.08143391 -0.03115265
  0.1263195  -0.02169068 -0.03204576 -0.04972161 -0.05322499 -0.00502034
 -0.03719112  0.01568306 -0.03014068  0.02186434 -0.02287652 -0.0159339
  0.00722756  0.03004224 -0.04827759 -0.0614485   0.03017768 -0.08580622
 -0.01084903  0.06604839 -0.0515257  -0.05936883 -0.00745368 -0.02845806
 -0.0596766   0.06106602 -0.04884266  0.0516535   0.02231757  0.04474395
  0.05824555  0.00748909 -0.14473593 -0.0734789   0.05841653  0.0551538
  0.04733743  0.0

## Convert the Encodings at the Centroids of each Cluster to Words to get a sense what Cluster is about

In [44]:
centroid_words = []
p_words=[]
centroid_words_dict = {}
for i in range(0,len(centroids)):
    c = centroids[i]
    similar_words = model.similar_by_vector(c, topn=10)
    words = [word for word, similarity in similar_words]
    print(words)
    p_words.append(similar_words)
    centroid_words_dict[f"c{i}"] = words

['%_#F########_9v.jsn', 'By_Jonas_Elmerraji', 'UDBKL', '%_#F########_7v.jsn', '%_#F########_3v.jsn', 'Components_www.danaher.com', 'SFIO_Quote_Chart', 'sanctuary_DellaBianca', 'By_Elise_Viebeck', 'By_Laurelle_Gilbert']
['CNA_ir', 'By_Bluto_l', 'dpa_pr', 'hollyrpeete_@', 'CNA_jy', 'Bob_Ewegen_deputy', 'Find_Ashvegas_updates', 'THE_PAGE_STOPS', 'Donald_Luzzatto_editorial', 'jta']
['By_Elise_Viebeck', 'By_Jonas_Elmerraji', '%_#F########_3v.jsn', 'By_Tejas_Gunas', 'SECOND_SPOT_DISPATCHES_FROM', 'UDBKL', 'Components_www.danaher.com', 'Pacgen_Reports', 'By_Laurelle_Gilbert', "FAN_'S_ATTIC"]
['%_#F########_9v.jsn', 'By_Jonas_Elmerraji', 'By_Laurelle_Gilbert', '%_#F########_3v.jsn', 'By_Christine_Fenno', 'BY_CHERYL_LAVIN_cheryllavin@aol.com', 'By_Tejas_Gunas', 'By_Elise_Viebeck', 'SFIO_Quote_Chart', 'DIRECTORS_OF_CAPITAL_CORP.']
['By_Elise_Viebeck', 'UDBKL', 'By_Jonas_Elmerraji', 'By_Laurelle_Gilbert', 'SFIO_Quote_Chart', '%_#F########_9v.jsn', 'McGrath_RentCorp_Sets', 'By_Tejas_Gunas', 'Compo

The first iteration of this analysis produced "hot garabage" - I will instead sample the 5 posts nearest to the center of each centroid

In [63]:
data["cluster"] = kmeans.labels_

/var/folders/jh/7_7blzs94bs2rrfl07z2j84r0000gn/T/ipykernel_1508/3625996920.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["cluster"] = kmeans.labels_
/var/folders/jh/7_7blzs94bs2rrfl07z2j84r0000gn/T/ipykernel_1508/3625996920.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['distance'] = data.apply(lambda row: np.linalg.norm(row["mean_encoded"] - centroids[int(row["cluster"])]), axis=1)


In [ ]:
centroids = kmeans.cluster_centers_
data['distance'] = data.apply(lambda row: np.linalg.norm(row["mean_encoded"] - centroids[int(row["cluster"])]), axis=1)

In [73]:
# Calculate the distance from each point to its cluster centroid
closest_points = {i: [] for i in range(k)}
for i in range(k):
    # Points in cluster i
    
    points_in_cluster = data[data["cluster"] == i]
    
    closest_indices = points_in_cluster["distance"].nsmallest(7).index
    
    # Store the indices of the closest points
    closest_points[i] = closest_indices

# Printing the indices of the 7 closest points for each cluster
for cluster, indices in closest_points.items():
    print(f"Cluster {cluster} has closest indices: {indices}")

Index([   2,    3,    4,    9,   15,   26,   27,   36,   41,   42,
       ...
       3446, 3447, 3483, 3550, 3566, 3573, 3574, 3584, 3662, 3717],
      dtype='int64', length=1795)
Index([   3,    9,   15,   36,   42,   47,   75,  125,  141,  155,
       ...
       1906, 1920, 1927, 2126, 2167, 2238, 2396, 2468, 2487, 3574],
      dtype='int64', length=272)
272
1795
Index([   2,    3,    4,    9,   15,   26,   27,   36,   41,   42,
       ...
       3446, 3447, 3483, 3550, 3566, 3573, 3574, 3584, 3662, 3717],
      dtype='int64', length=1795)
Index([   4,   27,   73,   83,   96,   98,  110,  121,  122,  139,
       ...
       3376, 3436, 3446, 3447, 3483, 3550, 3566, 3573, 3662, 3717],
      dtype='int64', length=459)
459
1795
Index([   2,    3,    4,    9,   15,   26,   27,   36,   41,   42,
       ...
       3446, 3447, 3483, 3550, 3566, 3573, 3574, 3584, 3662, 3717],
      dtype='int64', length=1795)
Index([  51,  212,  252,  300,  395,  417,  454,  481,  556,  637,
       ...
      

In [74]:
print(len(data))
print(len(expanded_df))

lv = list(data.loc[closest_points[1]]["post_body"])


for post in lv:
    print(post)
    print()
    print()
    print("----NEXT POST-----")

1795
1795
Not me by the way 


----NEXT POST-----
[https://twitter.com/i/status/1782826724813656361](https://twitter.com/i/status/1782826724813656361)

How is any of this free mixing, dancing, playing the annoying dammi falastini song helping the people in Palestine? These "protests" are just a cesspool for killing time and a break from their monotonous lives. And they think they'll get Allah's help by doing so. What a bunch of libtards.


----NEXT POST-----
Sorry if this is the wrong place to post this but earlier I was on a plane and some ppl were talking about the protest for Palestine and someone yelled "yeah we don't put up with that bullshit in the south" and it sparked a memory. So I live in a small town in Alabama and a few years ago when the blm movement was very big, was the first time I ever and till this day have seen a protest in my town. now it was literally only like 5 ppl but we drove past the town square and they were standing on the corner just holding up signs, I the

In [76]:
lv = list(data.loc[closest_points[i]]["post_body"])

for i in range(0,8):
    post_list = list(data.loc[closest_points[i]]["post_body"])
    print(f"These are the posts for Cluster {i}")
    print()
    for i in range(0,len(post_list)):
        post = post_list[i]
        print(f"Post #{i+1}:")
        print(post)
        print()


These are the posts for Cluster 0

Post #1:
Preface: I am Jewish. I would not say I support Israel, but I do favor a two state solution. Sort of a "Why can't we all just get along?"

What is it that seems to have everyone so anti-Israel right now? I know there's always war in the middle east, but tensions have risen a ton as of late. Hamas attacked Israel in October, and I know that Israel is fighting back, often in less than ethical ways. But aren't Hamas and Palestine also committing crimes and being violent against Israelis? Why has the general attitude turned so anti-Israel in what's going on?

The people who answer questions here are awesome

Post #2:
Preface: I am Jewish. I would not say I support Israel, but I do favor a two state solution. Sort of a "Why can't we all just get along?"

What is it that seems to have everyone so anti-Israel right now? I know there's always war in the middle east, but tensions have risen a ton as of late. Hamas attacked Israel in October, and I kno

In [77]:
for i in range(0,8):
    post_list = list(data.loc[closest_points[i]]["post_body"])
    print(f"# of Posts in Cluster {i} = {len(post_list)}")

# of Posts in Cluster0 = 12
# of Posts in Cluster1 = 10
# of Posts in Cluster2 = 9
# of Posts in Cluster3 = 10
# of Posts in Cluster4 = 8
# of Posts in Cluster5 = 11
# of Posts in Cluster6 = 9
# of Posts in Cluster7 = 9


### I then fed the output and the following question to Chat-GPT:

### "Can you count the number of posts in each cluster as well as provide a summary of the common narrative or perspective about the Israel-Hamas war (or lack there of) in each cluster?"

### I asked Chat-GPT to count the number of tasks as a mean to verify that Chat-GPT was in fact identifying that each of the posts were part of their own cluster

### I saved a link to the chat here: https://chat.openai.com/share/a4fb1fe7-6dd5-47a5-8606-92b0edd62808

Cluster 0:
Number of Posts: 12
Common Narrative or Perspective:
Cluster 0 mainly features posts from individuals identifying as Jewish, who express a nuanced stance on the Israel-Hamas conflict. They generally favor a two-state solution and express concern about the rising anti-Israel sentiment. These posts question why there's significant backlash against Israel while acknowledging that both Hamas and Palestinian authorities commit acts of violence. There's a recurring theme of seeking understanding and questioning the widespread negative perceptions of Israel, particularly in the context of recent conflicts.

Cluster 1:
Number of Posts: 10
Common Narrative or Perspective:
This cluster includes posts with a critical view of activism and media representation of the Israel-Hamas conflict. It touches on the cynicism about protests and the effectiveness of social activism, with some skepticism about the motivations behind pro-Palestinian protests, viewing them as superficial or disconnected from the actual needs of Palestinians. The narrative is somewhat disenchanted with the current forms of advocacy, which are seen as performative rather than genuinely supportive.

Cluster 2:
Number of Posts: 6
Common Narrative or Perspective:
Posts in Cluster 2 focus intensely on criticism of Israel's military actions and policies, highlighting alleged war crimes and human rights abuses by the IDF. There's a strong sentiment that Israeli actions are disproportionate and overly aggressive, contributing to a negative image of Israel on the global stage. The narrative is highly critical of Israel's justification for its military strategies and its impact on Palestinian civilians.

Cluster 3:
Number of Posts: 10
Common Narrative or Perspective:
Cluster 3 diverges from direct discussion of the Israel-Hamas conflict, instead focusing on broader geopolitical issues and personal anecdotes unrelated to the conflict. When the conflict is mentioned, the perspective is more about questioning or trying to understand the broader geopolitical impacts and historical contexts, rather than direct commentary on current events.

Cluster 4:
Number of Posts: 6
Common Narrative or Perspective:
This cluster contains critical perspectives on both sides of the conflict, with a significant emphasis on the moral and ethical implications of support for either Israel or Palestine. The posts often discuss the complexity of the conflict, acknowledging faults and extremist actions on both sides. There's a call for a more balanced view that avoids demonizing either side, with a strong push towards recognizing the nuances and challenges of achieving peace.

Cluster 5:
Number of Posts: 10
Common Narrative or Perspective:
Cluster 5 mainly focuses on global job postings and has little to no relevance to the Israel-Hamas conflict. It seems to be incorrectly categorized if meant to reflect discussions on the conflict.

Cluster 6:
Number of Posts: 9
Common Narrative or Perspective:
The narrative in Cluster 6 is heavily focused on exploring solutions and future scenarios post-conflict, discussing various peace plans and their feasibility. There's an analytical approach to what might work or not in achieving peace, with considerations of both Israeli and Palestinian perspectives. The posts often debate the practicality and moral implications of proposed solutions to the conflict.

Cluster 7:
Number of Posts: 9
Common Narrative or Perspective:
Posts in Cluster 7 express deep skepticism about the intentions and actions of political leaders on both sides, with a particular focus on Israeli politics. There's a strong sentiment of distrust towards Netanyahu and concerns about democratic backsliding in Israel. The narrative includes fears about Israel turning into a dictatorship under current leadership, with calls for radical changes or even revolution to ensure peace and democratic integrity.

These summaries reflect a range of perspectives on the Israel-Hamas conflict, from calls for understanding and peace to deep criticism of actions taken by both sides. Each cluster captures a different facet of the broader discourse surrounding this enduring and complex conflict.

### I then asked Chat-GPT 4 to summarize the narratives in a One-Line Sentence using this prompt: "Can you summarize each of those narratives into a one line sentence?"

Cluster 0:
Jewish individuals expressing a nuanced stance favoring a two-state solution, concerned about the rising anti-Israel sentiment amidst ongoing conflict.

Cluster 1:
Critiques of activism, questioning the authenticity and effectiveness of pro-Palestinian protests viewed as superficial or performative.

Cluster 2:
Strong criticisms of Israel's military actions, highlighting perceived war crimes and human rights abuses against Palestinians.

Cluster 3:
Discussions veering away from direct conflict narratives to explore broader geopolitical impacts and personal anecdotes unrelated to the conflict.

Cluster 4:
Balanced criticisms of both sides, advocating for moral reflection and avoidance of extreme positions in the Israel-Palestine conflict.

Cluster 5:
Primarily global job postings, not directly related to the Israel-Hamas conflict.

Cluster 6:
Analytical discussions on potential peace solutions and their feasibility, exploring future scenarios and peace plans.

Cluster 7:
Distrust towards political leadership in Israel, with fears of authoritarianism and calls for significant political reform or revolution.

### Drunk on the power of Chat-GPT 4 - I then kept going. I asked Chat-GPT 4 to produce the post that is the most emblematic of the narrative defined using the prompt
### "Can you produce the post for each cluster that is the most emblematic of the narrative defined?"

Cluster 0 - Nuanced Stance Favoring Two-State Solution
Post #1:
Preface: I am Jewish. I would not say I support Israel, but I do favor a two state solution. Sort of a "Why can't we all just get along?"
What is it that seems to have everyone so anti-Israel right now? I know there's always war in the middle east, but tensions have risen a ton as of late. Hamas attacked Israel in October, and I know that Israel is fighting back, often in less than ethical ways. But aren't Hamas and Palestine also committing crimes and being violent against Israelis? Why has the general attitude turned so anti-Israel in what's going on?

Cluster 1 - Critical View on Activism
Post #2:
https://twitter.com/i/status/1782826724813656361
How is any of this free mixing, dancing, playing the annoying dammi falastini song helping the people in Palestine? These "protests" are just a cesspool for killing time and a break from their monotonous lives. And they think they'll get Allah's help by doing so. What a bunch of libtards.

Cluster 2 - Criticisms of Israeli Military Actions
Post #3:
It's crazy to me that Israel went to be known as this tourist hub known for beaches and clubs and this reputation for human rights and democracy. A destination for many and example for all especially to the middle-east to basically a laughing stock and being revealed as a morally bankrupt state.

Cluster 3 - Broader Geopolitical Impact
Post #6:
Hi! My mom’s birthday is coming up and she is currently in Israel. I want to send some birthday present for her but don’t know how. I am from the Philippines. Does anyone know trusted websites/app to order cakes or flowers?

Cluster 4 - Balanced Criticisms of Both Sides
Post #1:
I said what I said.
I can’t fathom why people could so strongly support either the state of Israel or a Hamas-led Palestinian state. Both have been committing unspeakable acts against each other.

Cluster 5 - Global Job Postings
Post #1:
Company: Natera
Location: Afghanistan Albania Algeria Andorra Angola Antarctica Armenia Austria Azerbaijan Bahrain Belarus Belgium Benin Bosnia and Herzegovina Botswana Bouvet Island Bulgaria Burkina Faso Burundi Cabo Verde Cameroon Central African Republic Chad Comoros Congo Congo, The Democratic Republic of the Cook Islands Croatia Curaçao Cyprus Czechia Côte d'Ivoire Denmark Djibouti Egypt Equatorial Guinea Eritrea Estonia Eswatini Ethiopia Faroe Islands Finland France French Guiana French Southern Territories Gabon Gambia Georgia Germany Ghana Gibraltar Greece Greenland Guadeloupe Guernsey Guinea Guinea-Bissau Heard Island and McDonald Islands Holy See (Vatican City State) Hungary Iceland Iran Iraq Ireland Isle of Man Israel Italy Jersey Jordan Kazakhstan Kenya Kuwait Kyrgyzstan Latvia Lebanon Lesotho Liberia Libya Liechtenstein Lithuania Luxembourg Madagascar Malawi Mali Malta Martinique Mauritania Mauritius Mayotte Moldova Monaco Montenegro Morocco Mozambique Namibia Netherlands Niger Nigeria North Macedonia Norway Oman Palestine, State of Poland Portugal Qatar Romania Russian Federation Rwanda Réunion Saint Barthélemy Saint Helena, Ascension and Tristan da Cunha Saint Martin (French part) Saint Pierre and Miquelon San Marino Sao Tome and Principe Saudi Arabia Senegal Serbia Seychelles Sierra Leone Sint Maarten (Dutch part) Slovakia Slovenia Somalia South Africa South Sudan Spain Sudan Svalbard and Jan Mayen Sweden Switzerland Syrian Arab Republic Tajikistan Tanzania Togo Tunisia Turkey Turkmenistan Uganda Ukraine United Arab Emirates United Kingdom Uzbekistan Western Sahara Yemen Zambia Zimbabwe Åland Islands 📍

Cluster 6 - Peace Solutions and Their Feasibility
Post #2:
Maintaining the status quo (Self-evident)
Why it might work- Neither side can agree on a solution, and forcing a solution will lead to conflict.
Why it might not- Things are getting worse, not better. Tensions and attacks on both sides will only keep increasing.

Cluster 7 - Distrust Towards Political Leadership
Post #1:
Hi, I'm Israeli and, 6 months after Oct. 7, I wanted to share my opinions on all this mess. I am far-left, and what I am about to say at the end will probably seem extremist and insane, but I really ask you to read the whole thing so that you understand the whole picture I'm trying to pain here.

These posts each represent the defined narrative of their respective cluster, highlighting the dominant themes and concerns expressed within each group.

In [79]:
narrative_choice = [0,1,2,5,0,0,1,0]
narratives = []
for i in range(0,8):
    post_list = list(data.loc[closest_points[i]]["post_body"])
    narrative = post_list[narrative_choice[i]]
    print(f"------------------------------------- Cluster {i} -------------------------------------")
    print(narrative)
    narratives.append(narrative)

------------------------------------- Cluster 0 -------------------------------------
Preface: I am Jewish. I would not say I support Israel, but I do favor a two state solution. Sort of a "Why can't we all just get along?"

What is it that seems to have everyone so anti-Israel right now? I know there's always war in the middle east, but tensions have risen a ton as of late. Hamas attacked Israel in October, and I know that Israel is fighting back, often in less than ethical ways. But aren't Hamas and Palestine also committing crimes and being violent against Israelis? Why has the general attitude turned so anti-Israel in what's going on?

The people who answer questions here are awesome
------------------------------------- Cluster 1 -------------------------------------
[https://twitter.com/i/status/1782826724813656361](https://twitter.com/i/status/1782826724813656361)

How is any of this free mixing, dancing, playing the annoying dammi falastini song helping the people in Palestine?